# Residual Gated Graph ConvNets
### Xavier Bresson, Jan. 15 2018

In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import pdb 
import time
import numpy as np
import pickle

if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    #torch.cuda.manual_seed(1)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    #torch.manual_seed(1)
    
# import files in folder util
import sys
sys.path.insert(0, 'util/')
import block 
import graph_generator as g


from sklearn.metrics import confusion_matrix

cuda available


In [2]:
# semi-supervised clustering
if 1==1:
    task_parameters = {}
    task_parameters['flag_task'] = 'clustering'
    task_parameters['nb_communities'] = 10
    task_parameters['nb_clusters_target'] = task_parameters['nb_communities']
    task_parameters['Voc'] = task_parameters['nb_communities'] + 1
    task_parameters['size_min'] = 2
    task_parameters['size_max'] = 5
    task_parameters['p'] = 0.5
    task_parameters['q'] = 0.1  
    file_name = 'data/set_100_clustering_maps_p05_q01_size5_25_2017-10-31_10-25-00_.txt'
    with open(file_name, 'rb') as fp:
        all_trainx = pickle.load(fp)
    task_parameters['all_trainx'] = all_trainx[:100]


In [23]:
import sys
sys.path.insert(0, 'data/ppi/')
import process_ppi

train_adj,val_adj,test_adj,train_feat,val_feat,test_feat,train_labels,val_labels, test_labels, train_nodes, val_nodes, test_nodes, tr_msk, vl_msk, ts_msk = process_ppi.process_p2p()


Loading G...
5
directed
graph
nodes
links
multigraph
Loading id_map...
56944
56944
Loading features...
(56944, 50)
Loading class_map...
56944
Splitting graph...
Re-arranging sub-graph IDs...
Are sub-graphs isolated?
True


In [40]:
train_adj.shape

(20, 3480, 3480)

In [46]:
train_start = []
train_end = []

In [47]:
for k in range(20):
    train_Estart = np.zeros((150000, 3480))
    train_Eend = np.zeros((150000, 3480))

    # converting adjacency matrix to edge-to-start vertex matrix
    count = 0
    adj = train_adj[k]
    for i in range(adj.shape[0]):
        for j in range(adj.shape[1]):
            if adj[i,j] == 1:
                train_Estart[count,i] = 1
                train_Eend[count,j] = 1
                count += 1


    print(k, count)
    train_Estart = train_Estart[:count]
    train_Eend = train_Eend[:count]
    train_start.append(train_Estart)
    train_end.append(train_Eend)

0 34997
1 32461
2 61868
3 67577
4 38845
5 21194
6 46851
7 72451
8 10925
9 107806


MemoryError: 

In [ ]:
count

In [4]:
train_feat = Variable(torch.FloatTensor(train_feat), requires_grad=False,volatile=True).cuda()
train_adj = Variable(torch.FloatTensor(train_adj), requires_grad=False,volatile=True).cuda()
train_labels = Variable(torch.FloatTensor(train_labels), requires_grad=False,volatile=True).cuda()

val_feat = Variable(torch.FloatTensor(val_feat), requires_grad=False).cuda()
test_feat = Variable(torch.FloatTensor(test_feat), requires_grad=False).cuda()
val_labels = Variable(torch.FloatTensor(val_labels), requires_grad=False).cuda()
test_labels = Variable(torch.FloatTensor(test_labels), requires_grad=False).cuda()
val_adj = Variable(torch.FloatTensor(val_adj), requires_grad=False).cuda()
test_adj = Variable(torch.FloatTensor(test_adj), requires_grad=False).cuda()

/home/jriosfyp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """Entry point for launching an IPython kernel.
/home/jriosfyp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/jriosfyp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:

##############################
# Class cell definition
##############################
class OurConvNetcell(nn.Module):
    
    def __init__(self, dim_in, dim_out):
        super(OurConvNetcell, self).__init__()

        # conv1
        self.Ui1 = nn.Linear(dim_in, dim_out, bias=False) 
        self.Uj1 = nn.Linear(dim_in, dim_out, bias=False)
        self.Vi1 = nn.Linear(dim_in, dim_out, bias=False) 
        self.Vj1 = nn.Linear(dim_in, dim_out, bias=False)  
        self.bu1 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        self.bv1 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        
        # conv2
        self.Ui2 = nn.Linear(dim_out, dim_out, bias=False) 
        self.Uj2 = nn.Linear(dim_out, dim_out, bias=False)
        self.Vi2 = nn.Linear(dim_out, dim_out, bias=False) 
        self.Vj2 = nn.Linear(dim_out, dim_out, bias=False)  
        self.bu2 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        self.bv2 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        
        # bn1, bn2
        self.bn1 = torch.nn.BatchNorm1d(dim_out)
        self.bn2 = torch.nn.BatchNorm1d(dim_out)
        
        # resnet
        self.R = nn.Linear(dim_in, dim_out, bias=False) 
            
        # init
        self.init_weights_OurConvNetcell(dim_in, dim_out, 1)
        
         
    def init_weights_OurConvNetcell(self, dim_in, dim_out, gain):
        
        # conv1
        scale = gain* np.sqrt( 2.0/ dim_in )
        self.Ui1.weight.data.uniform_(-scale, scale) 
        self.Uj1.weight.data.uniform_(-scale, scale) 
        self.Vi1.weight.data.uniform_(-scale, scale) 
        self.Vj1.weight.data.uniform_(-scale, scale) 
        scale = gain* np.sqrt( 2.0/ dim_out )
        self.bu1.data.fill_(0)
        self.bv1.data.fill_(0)
        
        # conv2
        scale = gain* np.sqrt( 2.0/ dim_out )
        self.Ui2.weight.data.uniform_(-scale, scale) 
        self.Uj2.weight.data.uniform_(-scale, scale) 
        self.Vi2.weight.data.uniform_(-scale, scale) 
        self.Vj2.weight.data.uniform_(-scale, scale) 
        scale = gain* np.sqrt( 2.0/ dim_out )
        self.bu2.data.fill_(0)
        self.bv2.data.fill_(0)
        
        # RN
        scale = gain* np.sqrt( 2.0/ dim_in )
        self.R.weight.data.uniform_(-scale, scale)  
            
            
    def forward(self, x, E_start, E_end):
        
        # E_start, E_end : E x V

        xin = x
        # conv1
        Vix = self.Vi1(x)  #  V x H_out
        Vjx = self.Vj1(x)  #  V x H_out
        x1 = torch.mm(E_end,Vix) + torch.mm(E_start,Vjx) + self.bv1  # E x H_out
        x1 = F.sigmoid(x1)
        Uix = self.Ui1(x)  #  V x H_out
        x2 = torch.mm(E_start, Uix)  #  V x H_out
        Ujx = self.Uj1(x)  #  V x H_out
        x = Uix + torch.mm(E_end.t(), x1*x2) + self.bu1 #  V x H_out
        # bn1
        x = self.bn1(x)
        # relu1
        x = F.relu(x)
        # conv2
        Vix = self.Vi2(x)  #  V x H_out
        Vjx = self.Vj2(x)  #  V x H_out
        x1 = torch.mm(E_end,Vix) + torch.mm(E_start,Vjx) + self.bv2  # E x H_out
        x1 = F.sigmoid(x1)
        Uix = self.Ui2(x)  #  V x H_out
        x2 = torch.mm(E_start, Uix)  #  V x H_out
        Ujx = self.Uj2(x)  #  V x H_out
        x = Uix + torch.mm(E_end.t(), x1*x2) + self.bu2 #  V x H_out
        # bn2
        x = self.bn2(x)
        # addition
        x = x + self.R(xin)
        # relu2
        x = F.relu(x)
            
        return x
        
        
        
        
        
##############################
# Class NN definition
##############################  
class Graph_OurConvNet(nn.Module):
    
    def __init__(self, net_parameters):
        
        super(Graph_OurConvNet, self).__init__()
        
        # parameters
        flag_task = task_parameters['flag_task']
        Voc = net_parameters['Voc']
        D = net_parameters['D']
        nb_clusters_target = net_parameters['nb_clusters_target']
        H = net_parameters['H']
        L = net_parameters['L']
        
        # vector of hidden dimensions
        net_layers = []
        for layer in range(L):
            net_layers.append(H)
      
        
        # CL cells
        # NOTE: Each graph convnet cell uses *TWO* convolutional operations
        net_layers_extended = [D] + net_layers # include embedding dim
        L = len(net_layers)
        list_of_gnn_cells = [] # list of NN cells
        for layer in range(L//2):
            Hin, Hout = net_layers_extended[2*layer], net_layers_extended[2*layer+2]
            list_of_gnn_cells.append(OurConvNetcell(Hin,Hout))
            
        # register the cells for pytorch
        self.gnn_cells = nn.ModuleList(list_of_gnn_cells)
              
        # fc
        Hfinal = net_layers_extended[-1]
        self.fc = nn.Linear(Hfinal,nb_clusters_target) 
        
        # init
        self.init_weights_Graph_OurConvNet(Voc,D,Hfinal,nb_clusters_target,1)
        
        # print
        print('\nnb of hidden layers=',L)
        print('dim of layers (w/ embed dim)=',net_layers_extended)      
        print('\n')
        
        # class variables
        self.L = L
        self.net_layers_extended = net_layers_extended      
        self.flag_task = flag_task
        
        
    def init_weights_Graph_OurConvNet(self, Fin_enc, Fout_enc, Fin_fc, Fout_fc, gain):

        scale = gain* np.sqrt( 2.0/ Fin_enc )
        #self.encoder.weight.data.uniform_(-scale, scale)  
        scale = gain* np.sqrt( 2.0/ Fin_fc )
        self.fc.weight.data.uniform_(-scale, scale)  
        self.fc.bias.data.fill_(0)  
    
            
    def forward(self, x, E_start, E_end):
        for layer in range(self.L//2 -1):
            gnn_layer = self.gnn_cells[layer]            
            x = gnn_layer(x,E_start,E_end) # V x Hfinal
            x = F.dropout(x, 0.5, training=self.training)
        gnn_layer = self.gnn_cells[self.L//2-1]            
        x = gnn_layer(x,E_start,E_end) # V x Hfinal
            
        # FC
        x = self.fc(x)
        return x
        
    def loss(self, y, y_target, weight):
        loss = nn.CrossEntropyLoss(weight=weight.type(dtypeFloat))(F.logsigmoid(y),y_target)
        return loss
            
    def update(self, lr):            
        update = torch.optim.Adam( self.parameters(), lr=lr )
        return update

    def update_learning_rate(self, optimizer, lr):
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        return optimizer
    
    def nb_param(self):

        return self.nb_param

In [ ]:
def get_cora_dataset():
    x = Variable(torch.from_numpy(cora_labels_train_x))
    y = Variable(torch.from_numpy(cora_labels))

    nb_classes = len(np.unique(cora_labels)) 
    E_start = Variable(torch.from_numpy(cora_Estart).float())
    E_end = Variable(torch.from_numpy(cora_Eend).float())
    
    return x.cuda(), y.cuda(), E_start.cuda(), E_end.cuda(), labels.shape[0]

def get_cora_testset():
    x = Variable(torch.from_numpy(cora_labels_test_x))
    train_y = Variable(torch.from_numpy(cora_labels))
    #labels = cora_labels
    nb_classes = len(np.unique(cora_labels_test_y)) 
    E_start = Variable(torch.from_numpy(cora_Estart).float())
    E_end = Variable(torch.from_numpy(cora_Eend).float())
    
    return x.cuda(), train_y.cuda(), E_start.cuda(), E_end.cuda(), labels.shape[0]

In [ ]:
def get_net_datasets(train_x):
    train_y = Variable( torch.LongTensor(train_x.target).type(dtypeLong) , requires_grad=False) 
    # labels = train_y.data.cpu().numpy()

    # signal
    x = train_x.signal  # V-dim
    x = Variable( torch.LongTensor(x).type(dtypeLong) , requires_grad=False)
    # graph operators -  Edge = start vertex to end vertex
    E_start = train_x.edge_to_starting_vertex # E_start = E x V mapping matrix from edge index to corresponding start vertex
    E_end   = train_x.edge_to_ending_vertex # E_end = E x V mapping matrix from edge index to corresponding end vertex
    E_start = torch.from_numpy(E_start.toarray()).type(dtypeFloat)
    E_end = torch.from_numpy(E_end.toarray()).type(dtypeFloat) 
    E_start = Variable( E_start , requires_grad=False) 
    E_end = Variable( E_end , requires_grad=False) 
    
    return x, train_y, E_start, E_end, labels.shape[0]

def calculate_loss(loss_criterion, nb_classes, labels, pred_y, cora=False):
    cluster_sizes = np.zeros(nb_classes)
    for r in range(nb_classes):
        cluster = np.where(labels==r)[0] # returns list of indices where label of node = r 
        cluster_sizes[r] = len(cluster) # update size of cluster labelled r
    weight = torch.zeros(nb_classes)
    for r in range(nb_classes):
        sumj = 0
        for j in range(nb_classes):
            if j!=r:
                sumj += cluster_sizes[j]
        weight[r] = sumj/ V 
    loss = loss_criterion(pred_y,labels,weight)
    return loss

def calculate_avg_accuracy(nb_classes, labels, pred_y, overall=False):
    S = labels.data.cpu().numpy()
    C = np.argmax(torch.nn.Softmax(dim=0)(pred_y).data.cpu().numpy() , axis=1)
    if not overall:
        # computes avg of accuracy over clusters
        CM = confusion_matrix(S,C).astype(np.float32)
        for r in range(nb_classes):
            cluster = np.where(labels==r)[0] 
            CM[r,:] /= cluster.shape[0] # divide by no. of elements in cluster r
        accuracy = np.sum(np.diag(CM))/ nb_classes
        return accuracy, CM
    else:
        # computes overall accuracy
        return np.sum(S==C)/S.shape[0], 0

def update_lr(net, optimizer, average_loss, average_loss_old, lr, decay_rate):
    if average_loss > 0.99* average_loss_old:
        lr /= decay_rate
    average_loss_old = average_loss
    return net.update_learning_rate(optimizer, lr)

In [ ]:
CORA = 1

task_parameters = {}
net_parameters = {}

if CORA:
    task_parameters['nb_communities'] = 7
    net_parameters['D'] = features.shape[1]
    net_parameters['H'] = 8
    net_parameters['L'] = 2
else:
    task_parameters['nb_communities'] = 10
    net_parameters['D'] = 50
    net_parameters['H'] = 50
    net_parameters['L'] = 10

# semi-supervised clustering
task_parameters['flag_task'] = 'clustering'
task_parameters['nb_clusters_target'] = task_parameters['nb_communities']
task_parameters['Voc'] = task_parameters['nb_communities'] + 1
task_parameters['size_min'] = 5
task_parameters['size_max'] = 25
task_parameters['p'] = 0.5
task_parameters['q'] = 0.1  
file_name = 'data/set_100_clustering_maps_p05_q01_size5_25_2017-10-31_10-25-00_.txt'
with open(file_name, 'rb') as fp:
    all_trainx = pickle.load(fp)
task_parameters['all_trainx'] = all_trainx[:100]

# network parameters
net_parameters['Voc'] = task_parameters['Voc']
net_parameters['nb_clusters_target'] = task_parameters['nb_clusters_target']

net = Graph_OurConvNet(net_parameters, CORA)
if torch.cuda.is_available():
    net.cuda()
# number of network parameters
nb_param = 0
for param in net.parameters():
    nb_param += np.prod(list(param.data.size()))
print('nb_param=',nb_param,' L=',net_parameters['L'])

In [ ]:
# optimization parameters
if CORA:
    max_iters = 500
    batch_iters = 10
    lr = 0.0003
else:
    max_iters = 5000
    batch_iters = 100
    lr = 0.00075
    
decay_rate = 1.25
nb_classes = task_parameters['nb_communities']

# Optimizer
optimizer = net.update(lr) 

#############
# loop over epochs
#############
t_start = time.time()
t_start_total = time.time()
average_loss_old = 1e10
running_train_acc = running_accuracy = running_conf_mat = running_total = running_loss = 0.0
tab_results = []

for iteration in range(max_iters):  # loop over the dataset multiple times
    # forward, loss
    net.train()
    if CORA:
        x, train_y, E_start, E_end, V = get_cora_dataset()
        features_x = Variable(torch.FloatTensor(features).type(dtypeFloat), requires_grad=False)
        y = net.forward(features_x, E_start, E_end)
        loss = calculate_loss(net.loss, nb_classes, train_y[:140], y[:140])
    else:
        train_x = g.graph_semi_super_clu(task_parameters) # gets a new graph dataset
        x, train_y, E_start, E_end, V = get_net_datasets(train_x)
        y = net.forward(x, E_start, E_end)    
        loss = calculate_loss(net.loss, nb_classes, train_y, y)
    
    # backward, update
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    # confusion matrix
    if CORA:
        net.eval()
        y_eval = net.forward(features_x, E_start, E_end)    
        
        # validation Set
        train_acc, train_CM = calculate_avg_accuracy(nb_classes, train_y[:140], y_eval[:140])
        accuracy, CM = calculate_avg_accuracy(nb_classes, train_y[140:500], y[140:500])
        running_train_acc += train_acc    
    else:
        accuracy, CM = calculate_avg_accuracy(nb_classes, train_y, y)
    running_conf_mat += CM
    running_accuracy += accuracy    
    running_loss += loss.data[0] # increment loss
    running_total += 1

    # learning rate, print results
    if not iteration%batch_iters:
        # confusion matrix and accuracy
        average_conf_mat = running_conf_mat/ running_total
        average_accuracy = running_accuracy/ running_total
        average_loss = running_loss/ running_total
        
        # update learning rate 
        optimizer = update_lr(net, optimizer, average_loss, average_loss_old, lr, decay_rate)

        # save intermediate results
        tab_results.append([iteration,average_loss,100* average_accuracy, time.time()-t_start_total])

        # print results
        print('\niteration= %d, loss(%diter)= %.3f, lr= %.8f, time(%diter)= %.2f' % 
              (iteration, batch_iters, average_loss, lr, batch_iters, time.time() - t_start))
        #print('Confusion matrix= \n', 100* average_conf_mat)
        print('accuracy= %.3f' % (100* average_accuracy))
        print('train accuracy= %.3f' % (100* running_train_acc/running_total))
        
        # reset counters
        t_start = time.time()
        running_train_acc = running_accuracy = running_conf_mat = running_total = running_loss = 0.0


In [ ]:
############            
# Evaluation on 100 pre-saved data
############
running_accuracy = running_conf_mat = running_total = running_loss = 0.0
if CORA:
    iters = 1
else:
    iters = 100

for iteration in range(iters):
    if CORA:
        x, labels, E_start, E_end, V = get_cora_dataset()
        features_x = Variable(torch.FloatTensor(features).type(dtypeFloat), requires_grad=False)
        y = net.forward(features_x, E_start, E_end)
        loss = calculate_loss(net.loss, nb_classes, labels[500:1500], y[500:1500])
    else:
        train_x = task_parameters['all_trainx'][iteration][1]
        x, labels, E_start, E_end, V = get_net_datasets(train_x)
        y = net.forward(x, E_start, E_end)
        loss = calculate_loss(net.loss, nb_classes, labels, y)
    
    running_loss += loss.data[0]
    running_total += 1

    # confusion matrix
    accuracy, CM = calculate_avg_accuracy(nb_classes, labels[500:1500], y[500:1500], overall=False)
    running_conf_mat += CM
    running_accuracy += accuracy    
    
# print results
average_accuracy = running_accuracy/ running_total
average_loss = running_loss/ running_total
print('\nloss(100 pre-saved data)= %.3f, accuracy(100 pre-saved data)= %.3f' % (average_loss,100* average_accuracy))